In [1]:
import numpy as np
import pandas as pd

# 1.数据透视表：

In [2]:
df = pd.DataFrame({"style":["one", "two", "three","one", "two", "three","one", "two", "three"],
                   "variable":["A","A","A","B","B","B","A","B","A"],
                   "label":["No","Yes","No","Yes","No","Yes","No","Yes","Yes"],
                   "value1":np.arange(1,10),
                   "value2":np.arange(10,19),
                   "value3":np.random.standard_normal(9)})
df

,style,variable,label,value1,value2,value3
0,one,A,No,1,10,1.630015
1,two,A,Yes,2,11,-0.273237
2,three,A,No,3,12,-0.405931
3,one,B,Yes,4,13,0.334020
4,two,B,No,5,14,0.461320
5,three,B,Yes,6,15,1.057019
6,one,A,No,7,16,-0.557411
7,two,B,Yes,8,17,0.487870
8,three,A,Yes,9,18,-0.596338


### 假设，想要计算一个一张在行方向上按 style 和 label 排列的分组平均值的表

In [3]:
df.pivot_table(index=["style","label"])

value1  value2    value3
style label                          
one   No        4.0    13.0  0.536302
      Yes       4.0    13.0  0.334020
three No        3.0    12.0 -0.405931
      Yes       7.5    16.5  0.230340
two   No        5.0    14.0  0.461320
      Yes       5.0    14.0  0.107317

In [4]:
# 也可以通过 droupby 实现
df.groupby(["style","label"]).mean()

value1  value2    value3
style label                          
one   No        4.0    13.0  0.536302
      Yes       4.0    13.0  0.334020
three No        3.0    12.0 -0.405931
      Yes       7.5    16.5  0.230340
two   No        5.0    14.0  0.461320
      Yes       5.0    14.0  0.107317

### 在行方向上按 style 和 variable 排列的分组，在列方向按 label 分组，只对 value1 和 value2 聚合

In [5]:
df.pivot_table(index=["style","variable"] , columns="label" , values=["value1","value2"])

value1      value2      
label              No  Yes     No   Yes
style variable                         
one   A           4.0  NaN   13.0   NaN
      B           NaN  4.0    NaN  13.0
three A           3.0  9.0   12.0  18.0
      B           NaN  6.0    NaN  15.0
two   A           NaN  2.0    NaN  11.0
      B           5.0  8.0   14.0  17.0

### 传递 margins=True 来扩充此表以包含部分总计

In [6]:
df.pivot_table(index=["style","variable"] , columns="label" , values=["value1","value2"] , margins=True)

value1           value2            
label              No  Yes  All     No   Yes   All
style variable                                    
one   A           4.0  NaN  4.0   13.0   NaN  13.0
      B           NaN  4.0  4.0    NaN  13.0  13.0
three A           3.0  9.0  6.0   12.0  18.0  15.0
      B           NaN  6.0  6.0    NaN  15.0  15.0
two   A           NaN  2.0  2.0    NaN  11.0  11.0
      B           5.0  8.0  6.5   14.0  17.0  15.5
All               4.0  5.8  5.0   13.0  14.8  14.0

### 要使用均值以外的聚合函数，请将其传递给 aggfunc 关键字参数

In [7]:
df.pivot_table(index=["style","variable"] , columns="label" , values=["value1","value2"] ,aggfunc=sum , margins=True)

value1           value2           
label              No   Yes All     No   Yes  All
style variable                                   
one   A           8.0   NaN   8   26.0   NaN   26
      B           NaN   4.0   4    NaN  13.0   13
three A           3.0   9.0  12   12.0  18.0   30
      B           NaN   6.0   6    NaN  15.0   15
two   A           NaN   2.0   2    NaN  11.0   11
      B           5.0   8.0  13   14.0  17.0   31
All              16.0  29.0  45   52.0  74.0  126

### fill_value,填充NA值

In [8]:
df.pivot_table(index=["style","variable"] , columns="label" , values=["value1","value2"] ,aggfunc=sum , margins=True , fill_value=99)

value1         value2         
label              No Yes All     No Yes  All
style variable                               
one   A             8  99   8     26  99   26
      B            99   4   4     99  13   13
three A             3   9  12     12  18   30
      B            99   6   6     99  15   15
two   A            99   2   2     99  11   11
      B             5   8  13     14  17   31
All                16  29  45     52  74  126

# 2.交叉表：

In [9]:
df = pd.DataFrame({"style":["one", "two", "three","one", "two", "three","one", "two", "three"],
                   "variable":["A","A","A","B","B","B","A","B","A"],
                   "label":["No","Yes","No","Yes","No","Yes","No","Yes","Yes"],
                   "value1":np.arange(1,10),
                   "value2":np.arange(10,19)})
df

,style,variable,label,value1,value2
0,one,A,No,1,10
1,two,A,Yes,2,11
2,three,A,No,3,12
3,one,B,Yes,4,13
4,two,B,No,5,14
5,three,B,Yes,6,15
6,one,A,No,7,16
7,two,B,Yes,8,17
8,three,A,Yes,9,18


### 按照style列和variable列来总结数据

In [10]:
df.pivot_table(index="style" , columns="variable" , values="value1" ,aggfunc="count" , margins=True)

variable,A,B,All
style,,,
one,2,1,3
three,2,1,3
two,1,2,3
All,5,4,9


In [11]:
pd.crosstab(index=df["style"] , columns=df["variable"] , margins=True)

variable,A,B,All
style,,,
one,2,1,3
three,2,1,3
two,1,2,3
All,5,4,9


### crosstab 的前两个参数都可以是数组或Series 或数组列表

In [12]:
pd.crosstab(index=[df["style"],df["label"]] , columns=df["variable"] , margins=True)

variable     A  B  All
style label           
one   No     2  0    2
      Yes    0  1    1
three No     1  0    1
      Yes    1  1    2
two   No     0  1    1
      Yes    1  1    2
All          5  4    9

In [13]:
df.pivot_table(index=["style","label"] , columns="variable" , values="value1" ,aggfunc="count" , margins=True , fill_value=0)

variable     A  B  All
style label           
one   No     2  0    2
      Yes    0  1    1
three No     1  0    1
      Yes    1  1    2
two   No     0  1    1
      Yes    1  1    2
All          5  4    9